# Support Vector Regression

In [1]:
%matplotlib inline
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_columns = None
pd.options.display.max_rows = None

# set seeds
np.random.seed(1)
random.seed(1)

# path data
path_data = '../data/'

# Clean data

## Create and drop features, manage categorical

In [2]:
# Function to prepare data

from sklearn.base import TransformerMixin

def hex2rgb(hex_code):
    return np.array([int(hex_code[i:i+2], 16) for i in (0, 2, 4)])

def hex2dominantChannel(hex_code, label=None):
    # If nan or not an hexadecimal
    if type(hex_code) == float:
        return np.nan
    colors_map = {0: 'red', 1: 'green', 2: 'blue'}
    rgb = hex2rgb(hex_code)
    y = np.argmax(rgb)
    return label + " " + colors_map[y] if label is not None else colors_map[y]

def create_new_features(X):
    # transform utc offset:
    X['Sin_UTC'] = np.sin((11 * 3600 + X['UTC Offset']) / (24 * 3600) * 2 * np.pi)
    X['Cos_UTC'] = np.cos((11 * 3600 + X['UTC Offset']) / (24 * 3600) * 2 * np.pi)
    
    # time since creation in days
    duration = (pd.to_datetime('today') - 
                pd.to_datetime(X['Profile Creation Timestamp']).dt.tz_localize(None))
    X['Duration'] = duration.apply(lambda x: x.days)
    
    # convert personal url into True/False (NaNs or unique)
    X['Has Personal URL'] = X['Personal URL'].notna()
    
    # Group color into Red, Blue, and Green
    X['Profile Text Color'] = df['Profile Text Color'].apply(lambda x: hex2dominantChannel(x, label="text"))
    X['Profile Theme Color'] = df['Profile Theme Color'].apply(lambda x: hex2dominantChannel(x, label="theme"))
    X['Profile Page Color'] = df['Profile Page Color'].apply(lambda x: hex2dominantChannel(x, label="page"))
    
    # log(x + 1) of numerical features
    for feature in ['Num of Followers', 'Num of People Following',
                    'Num of Status Updates', 'Num of Direct Messages',
                    'Avg Daily Profile Visit Duration in seconds',
                    'Avg Daily Profile Clicks']:
        X[f'log {feature}'] = np.log1p(X[feature])
    
    return X
    
def clean_features(X):
    # merge categories names with and without cap letter
    X['Location Public Visibility'] = X['Location Public Visibility'].str.lower()
    X.loc[X['Profile Category'] == " ", 'Profile Category'] = 'unknown'
    return X

def drop_features(X, l_features):
    return X.drop(columns=l_features)
    
class RemoveCategories(TransformerMixin):
    def __init__(self, min_obs=10, l_cols=[]):
        self.min_obs = min_obs
        self.l_cats = {}
        self.l_cols = l_cols

    def fit(self, X):
        # assumes all columns of df_cat are strings
        for col in self.l_cols:
            val_counts = X[col].fillna('missing').value_counts()
            self.l_cats[col] = val_counts[val_counts >= self.min_obs].index.tolist()
        return self

    def transform(self, X):
        # assumes X is a DataFrame
        for col in self.l_cols:
            X.loc[~ X[col].isin(self.l_cats[col]), col] = 'other'        
        return X

In [3]:
# data cleaning and feature engineering

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

l_cat_small = ['Profile Text Color', 'Profile Page Color',
               'Profile Theme Color', 'User Language', 'Location',
               'Profile Image Dominant Color', 'Profile Image Avatar (18)']
l_drop = ['Id', 'User Name', 'Profile Image', 'Personal URL',
          'Profile Creation Timestamp', 'Location', 'UTC Offset']
l_drop_logs = ['Num of Followers', 'Num of People Following',
               'Num of Status Updates', 'Num of Direct Messages',
               'Avg Daily Profile Visit Duration in seconds', 
               'Avg Daily Profile Clicks']

features_to_drop =dict(l_features = l_drop + l_drop_logs)

pipe = Pipeline([('create new features',
                  FunctionTransformer(create_new_features)),
                 #('transform color features', FunctionTransformer(transform_color_features)),
                 ('clean data', FunctionTransformer(clean_features)),
                 ('remove categories', RemoveCategories(min_obs=10, 
                                                        l_cols=l_cat_small)),
                 ('drop variables', FunctionTransformer(drop_features,
                                    kw_args=features_to_drop))
                ])

## Imputation and encoding/scaling

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

# numerical features
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
    ])

# categorical features
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(sparse=False, handle_unknown='ignore'))])

In [5]:
# train data
im_df = pd.read_csv('../data/train_profile_images_data.csv')
df = pd.read_csv('../data/train.csv')
df[['Profile Image Avatar (18)', 'Profile Image Dominant Color']] = im_df[['Profile Image Avatar (18)','Profile Image Dominant Color']]

y = np.log1p(df['Num of Profile Likes'].values)
X = df.drop(columns='Num of Profile Likes')

X = pipe.fit_transform(X)

preprocessor = ColumnTransformer(
    transformers=[('num', numeric_transformer,
                   X.select_dtypes(exclude=['object', 'bool']).columns),
                  ('cat', categorical_transformer,
                   X.select_dtypes(include=['object', 'bool']).columns)])

X = preprocessor.fit_transform(X)
X.shape

FileNotFoundError: [Errno 2] No such file or directory: '../data/train_profile_images_data.csv'

In [ ]:
# to get the columns names back

from sklearn.feature_extraction.text import _VectorizerMixin
from sklearn.feature_selection._base import SelectorMixin
def get_feature_out(estimator, feature_in):
    if hasattr(estimator,'get_feature_names'):
        if isinstance(estimator, _VectorizerMixin):
            # handling all vectorizers
            return [f'vec_{f}' for f in estimator.get_feature_names()]
        else:
            return estimator.get_feature_names(feature_in)
    elif isinstance(estimator, SelectorMixin):
        return np.array(feature_in)[estimator.get_support()]
    else:
        return feature_in
def get_ct_feature_names(ct):
    # handles all estimators, pipelines inside ColumnTransfomer
    # doesn't work when remainder =='passthrough'
    # which requires the input column names.
    output_features = []
    for name, estimator, features in ct.transformers_:
        if name!='remainder':
            if isinstance(estimator, Pipeline):
                current_features = features
                for step in estimator:
                    current_features = get_feature_out(step, current_features)
                features_out = current_features
            else:
                features_out = get_feature_out(estimator, features)
            output_features.extend(features_out)
        elif estimator=='passthrough':
            output_features.extend(ct._feature_names_in[features])
    return output_features

df_X = pd.DataFrame(X, columns=get_ct_feature_names(preprocessor))

In [ ]:
df_X.info()

In [ ]:
df_X.head()

# XGBoost

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor

gb= GradientBoostingRegressor(random_state=0)

parameters = {'n_estimators': [200, 300, 400],
              'n_iter_no_change': [6, 8, 10],
              'learning_rate': [0.02, 0.05, 0.10],
              'min_samples_split': range(2, 10),
              'min_samples_leaf': range(2, 10),
              'max_depth': range(2, 10)}

reg_gb = RandomizedSearchCV(gb, parameters, scoring='neg_mean_squared_error',
                            n_jobs=50, random_state=None, n_iter=100, verbose=10)
reg_gb.fit(X, y)

print(f'RMSLE: {np.sqrt(-reg_gb.best_score_):.3f}',
      f'\nbest parameters: { reg_gb.best_params_}')

In [ ]:
stop

# Submission

In [ ]:
def create_submission(estimator, number):
    X_test = pd.read_csv('data/test.csv')
    
    im_df = pd.read_csv('data/test_profile_images_data.csv')
    df[['Profile Image Avatar (18)', 'Profile Image Dominant Color']] = im_df[['Profile Image Avatar (18)','Profile Image Dominant Color']]
    
    Id = X_test['Id']
    X_test = pipe.transform(X_test)
    X_test = preprocessor.transform(X_test)
    y_pred = (estimator.predict(X_test))
    y_pred_test = pd.Series(np.expm1(y_pred))
    submission = pd.DataFrame({'Id': Id, 'Predicted': y_pred_test})
    path = f'submissions/submission{number}.csv'
    while os.path.exists(path):
        number += 1
        path = f'submissions/submission{number}.csv'
    submission.to_csv(path, index=False)
    print(f'submission #{number} created')